In [ ]:
using Pkg
Pkg.instantiate()
using DrugResponseModel
using BlackBoxOptim

# import data from the path
conc_l, pop, g2, g1, g2_0, g1_0 = setup_data("lapatinib"); # in which:

""" Cost function allowing the number of cell cycle species to change. """
function fullHillCost(hillParams)
    nG1 = floor(hillParams[end-1])
    nG2 = floor(hillParams[end])
    
    return DrugResponseModel.residHill(hillParams[1:end-2], conc_l, g1, g2, g1_0, g2_0, Int(nG1), Int(nG2))
end

# pop: population data
# g1, g2: g1 and g2 data
# initial: initial number of cells in g1 and in g2 at time 0

lowEC50 = 10.0
highEC50 = 500.0

low =  [lowEC50,  0.01, 0.01, 0.1, 0.01, 0.01, lowEC50,  0.0, 0.1, 0.0,  1,  1];
high = [highEC50, 3.0, 3.0, 3.0, 3.0, 3.0, highEC50, 3.0, 3.0, 3.0, 90, 90];

In [ ]:
results_ode = bboptimize(fullHillCost; SearchRange=collect(zip(low, high)),
                                       NumDimensions=length(low),
                                       TraceMode=:verbose,
                                       TraceInterval=20,
                                       MaxSteps=1E5);